In [ ]:
import os
import time
import math
import re
import struct
import traceback
import codecs

import bitstring

import numpy as np
import matplotlib.pyplot as plt
# import cv2
import pandas as pd

from PIL import Image
import IPython.display as display

from JIS0208 import JIS0208

In [ ]:
os.listdir('.')

In [ ]:
file_list = [
    'ETL8B2C1',
    'ETL8B2C2',
    'ETL8B2C3',
]

In [ ]:
filename = file_list[0]
print(filename)
file_stream = bitstring.ConstBitStream(filename=filename)

**The first record of each file is zero padded dummy.**

In [ ]:
record = file_stream.readlist('int:16,uint:16,uint:32,bytes:504')
record_dict = {
    'sheet_number': record[0], # type: int - Serial Sheet Number
    'JIS_KANJI_CODE': record[1], # type: binary - JIS Kanji Code (JIS X 0208)
    'JIS_READING': record[2], # type: ASCII - JIS Typical Reading (e.g. 'AI.M')
    'image_data': record[3], # type: bytes
}

width = 64
height = 63

np_img = np.array(Image.frombytes('1', (width, height), record_dict['image_data'], 'raw')).astype(int)
plt.imshow(np_img)
plt.colorbar()

In [ ]:
record = file_stream.readlist('int:16,uint:16,uint:32,bytes:504')
record_dict = {
    'sheet_number': record[0], # type: int - Serial Sheet Number
    'JIS_KANJI_CODE': record[1], # type: binary - JIS Kanji Code (JIS X 0208)
    'JIS_READING': record[2], # type: ASCII - JIS Typical Reading (e.g. 'AI.M')
    'image_data': record[3], # type: bytes
}

width = 64
height = 63

np_img = np.array(Image.frombytes('1', (width, height), record_dict['image_data'], 'raw')).astype(int)
plt.imshow(np_img)
plt.colorbar()

In [ ]:
hex(record_dict['JIS_KANJI_CODE'])

In [ ]:
total_samples = 0
record_count = {}

for filename in file_list:

    file_stream = bitstring.ConstBitStream(filename=filename)
    # skip the first record in each file
    file_stream.readlist('bytes:512')

    while True:
        try:
            record = file_stream.readlist('int:16,int:16,uint:32,bytes:504')
        except:
            break
        
        total_samples += 1
        record_dict = {
            'sheet_number': record[0], # type: int - Serial Sheet Number
            'JIS_CODE': record[1], # type: binary - JIS Kanji Code (JIS X 0208)
            'JIS_READING': record[2], # type: ASCII - JIS Typical Reading (e.g. 'AI.M')
            'image_data': record[3], # type: bytes
        }

        jis_code = record_dict['JIS_CODE']
        
        if jis_code in JIS0208.keys():
            unicode_char = JIS0208[jis_code]
        else:
            unicode_char = 'JIS_CODE=' + repr(jis_code)

        if unicode_char in record_count.keys():
            record_count[unicode_char] += 1
        else:
            record_count[unicode_char] = 1

In [ ]:
total_samples

In [ ]:
len(record_count.keys())

In [ ]:
record_count

In [ ]:
csv_filename = 'classes.tsv'
with open(csv_filename, mode='w', encoding='utf-8') as f:
    f.write('class\tnum_samples\n')
    for k in record_count.keys():
        log_str = f'{k}\t{record_count[k]}\n'
        f.write(log_str)
# sort classes
pd_df = pd.read_csv(csv_filename, encoding='utf-8', sep='\t')
pd_df = pd_df.sort_values(['class'])
pd_df.to_csv(csv_filename, encoding='utf-8', index=False, sep='\t')